In [1]:
try:
    import transformers
except ImportError:
    !pip install transformers
    import transformers
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 12.4 MB/s 
     |████████████████████████████████| 101 kB 3.5 MB/s 
     |████████████████████████████████| 6.6 MB 38.0 MB/s 
     |████████████████████████████████| 596 kB 38.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 9.3 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=b7c0c216d1e0f7153705bad2f7fd3dfb0eee223e73ac16f518e0d4ae1dc976a0
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [2]:
local_run = False
local_path = "C:\\Greenwich\\MSc Project\\project_code\\"
store_local = False

In [9]:
import pandas as pd
import numpy as np

import random
import string
import re
import ast

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

if local_run:
    nltk.data.path.append('/Users/algin/VOLD/nltk_data')
    nltk.download('stopwords',download_dir='/Users/algin/VOLD/nltk_data')
    nltk.download('punkt',download_dir='/Users/algin/VOLD/nltk_data')
else:
    nltk.download('stopwords')
    nltk.download('punkt')
stop_words = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import TFDistilBertForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

from sklearn.metrics import confusion_matrix,f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_train_data.csv?raw=true")
# test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_test_data.csv?raw=true")

In [5]:
# train_data, val_data = train_test_split(full_train_data,test_size=5000,random_state=21)

In [6]:
full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/uniform/uniform_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/uniform/uniform_test_data.csv?raw=true")

In [7]:
len(full_train_data), len(test_data)

(91269, 9851)

In [10]:
split = StratifiedShuffleSplit(n_splits=2,test_size=0.1, random_state=23)
for train_index, val_index in split.split(full_train_data[['tweet_text','cleaned']],full_train_data['is_cyberbullying']):
    train_data = full_train_data.loc[train_index]
    val_data = full_train_data.loc[val_index]

In [11]:
len(train_data), len(val_data), len(test_data)

(82142, 9127, 9851)

**Splitting into labels and features**

In [12]:
train_data.head()

,tweet_text,is_cyberbullying,cleaned
8163,@iamlabeng peace and order at the price of HR ...,1,peac order price hr violat airport lgu achiev ...
71649,"Foo Fighters edit \n\nIt wasn't disruptive, I ...",0,foo fighter edit disrupt ask valid question ex...
29745,and make a video with screenshots exposing my ...,1,make video screenshot expos bulli bulli high s...
10622,#Feminazi's gone wild! Smh!😒,1,feminazi gone wild smh
76699,"I agree. More than one overlapping articles, p...",0,agre one overlap articl practic theme neg effe...


In [13]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

In [14]:
banned_list= string.punctuation
punctuation_reg_exp = "[" + banned_list + "]"
emoji_reg_exp = get_emoji_regexp()

def clean_text(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)

    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    
    text = re.sub(punctuation_reg_exp,"",text)

    text = re.sub("\s\s+" , " ", text)

    text = re.sub(emoji_reg_exp, r"", text)

    return text

In [15]:
def split_label_and_feature(data):
    data['cleaned'] = data['tweet_text'].apply(clean_text)
    return data['cleaned'], data['is_cyberbullying']

In [16]:
X_train, y_train = split_label_and_feature(train_data)
X_val, y_val = split_label_and_feature(val_data)
X_test, y_test = split_label_and_feature(test_data)

In [17]:
X_train.head()

8163     peace order price hr violations airport lgu ac...
71649    foo fighters edit disruptive asking valid ques...
29745    make video screenshots exposing bullies bullie...
10622                              feminazis gone wild smh
76699    agree one overlapping articles practical theme...
Name: cleaned, dtype: object

**Model Training**

In [18]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

MODEL_NAME = 'distilbert-base-uncased'
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_layer_norm', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'dropout_19', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [19]:
def create_tf_dataset(X_feature,X_label):
    encodings = tokenizer(list(X_feature.values),truncation=True,padding=True)
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings),list(X_label.values)))
    return dataset

In [20]:
train_dataset = create_tf_dataset(X_train, y_train)

In [21]:
val_datset = create_tf_dataset(X_val, y_val)

In [22]:
# for _layer in model.layers:
#     if _layer.name == 'distilbert':
#         print(f"Freezing model layer {_layer.name}")
#         _layer.trainable = False
#     print(_layer.name)
#     print(_layer.trainable)
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________


In [23]:
BATCH_SIZE = 16
N_EPOCHS = 3

#chose the optimizer
optimizerr = tf.keras.optimizers.Adam(learning_rate=5e-5)
#define the loss function 
losss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#build the model
model.compile(optimizer=optimizerr,
              loss=losss,
              metrics=['accuracy'])
# train the model 
model.fit(train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE),
          epochs=N_EPOCHS,
          batch_size=BATCH_SIZE)

Epoch 1/3
5134/5134 [==============================] - 2942s 570ms/step - loss: 0.2394 - accuracy: 0.8982
Epoch 2/3
5134/5134 [==============================] - 2924s 570ms/step - loss: 0.1675 - accuracy: 0.9305
Epoch 3/3
5134/5134 [==============================] - 2924s 570ms/step - loss: 0.1092 - accuracy: 0.9564


**Saving Model**

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
cd drive/MyDrive/MSc\ Data\ Science/MSc\ Project/models

/content/drive/MyDrive/MSc Data Science/MSc Project/models


In [26]:
model.save_pretrained('uniform-distilbert')

In [27]:
def predict_proba(text_list, model, tokenizer):  
    #tokenize the text
    encodings = tokenizer(text_list, 
                          truncation=True, 
                          padding=True)
    #transform to tf.Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings)))
    #predict
    preds = model.predict(dataset.batch(1)).logits  
    
    #transform to array with probabilities
    res = tf.nn.softmax(preds, axis=1).numpy()      
    
    return res

In [28]:
predict_proba(["What a good day"],model,tokenizer)

array([[0.98842293, 0.01157708]], dtype=float32)

In [29]:
predict_proba(["Muslims are terrorists"],model,tokenizer)

array([[0.02215423, 0.9778457 ]], dtype=float32)

In [30]:
predict_proba(["You are an asshole"],model,tokenizer)

array([[0.04094362, 0.9590564 ]], dtype=float32)

In [31]:
predict_proba(["I believe in Christianity"],model,tokenizer)

array([[0.9386025 , 0.06139743]], dtype=float32)